### I. Library import 


In [127]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [128]:
pip install geodatasets

Note: you may need to restart the kernel to use updated packages.


In [129]:
import ipywidgets as widgets
from ipywidgets import interact, interactive,HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import geodatasets
import matplotlib.pyplot as plt

In [130]:
data2 = pd.read_csv("data.csv",low_memory=False)
data1 = pd.read_csv("data2023.csv",low_memory=False)
data = pd.concat([data1, data2], ignore_index=True)

In [131]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1584106 entries, 0 to 1584105
Data columns (total 16 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   date_mutation              1584106 non-null  object 
 1   nature_mutation            1584106 non-null  object 
 2   valeur_fonciere            1584106 non-null  float64
 3   code_commune               1584106 non-null  object 
 4   code_departement           1584106 non-null  object 
 5   id_parcelle                1584106 non-null  object 
 6   nombre_lots                1584106 non-null  int64  
 7   code_type_local            1584106 non-null  float64
 8   nombre_pieces_principales  1584106 non-null  float64
 9   surface_terrain            1584106 non-null  float64
 10  longitude                  1584106 non-null  float64
 11  latitude                   1584106 non-null  float64
 12  tranche_valeur             1584106 non-null  object 
 13  plage_surfac

### II. Création des fonctions

In [132]:
def selection_filtres_et_analyses(df):
    # **Préparation des filtres**
    type_local_map = {
        1: 'Maison',
        2: 'Appartement',
        3: 'Dépendance (isolée)',
        4: 'Local industriel et commercial ou assimilés'
    }
    df['type_local'] = df['code_type_local'].map(type_local_map)
    
    departements = df['code_departement'].unique()
    types_biens = df['type_local'].unique()
    
    # **Widgets pour les filtres**
    departement_select = widgets.SelectMultiple(
        options=departements,
        value=[],
        description='Départements',
        layout=widgets.Layout(width='60%')
    )
    departement_select.style.description_width = '100px'
    
    types_biens = df['type_local'].unique()
    type_bien_select = widgets.SelectMultiple(
        options=types_biens,
        value=[],
        description='Types de biens',
        layout=widgets.Layout(width='60%')
    )
    type_bien_select.style.description_width = '100px'

    tranche_prix = df['tranche_valeur'].unique()
    tranche_prix_select = widgets.SelectMultiple(
        options=tranche_prix,
        value=[],
        description='Tranche de prix',
        layout=widgets.Layout(width='60%')
    )
    tranche_prix_select.style.description_width = '100px'

    tranche_surface = df['plage_surface'].unique()
    tranche_surface_select = widgets.SelectMultiple(
        options=tranche_surface,
        value=[],
        description='Tranche surface',
        layout=widgets.Layout(width='60%')
    )
    tranche_surface_select.style.description_width = '100px'

    
    
    # **Widget pour sélectionner l'analyse à exécuter**
    analyses_disponibles = {
        "Prix moyen par m²": afficher_prix_moyen_par_m2,
        "Surface moyenne" : afficher_surface_moyenne_par_dept,
        "Evolution des prix" : evolution_prix_moyen_par_m2_par_departement_mensuel,
        "Nombre pièces principales" :afficher_nombre_piece_departement,
        "Type de bien par departement" : repartition_type_bien_par_departement,
        "Répartition des valeurs foncières" : repartition_Valeur_fonciere_par_tranche,
        "Prix m² en fonction de la surface" : prix_m2_par_surface,
        "Afficher la carte":afficher_carte_france
        
    }
    
    analyse_select = widgets.SelectMultiple(
        options=list(analyses_disponibles.keys()),
        description='Analyse',
        layout=widgets.Layout(width='60%')
    )
    
    analyse_select.style.description_width = '100px'
    
    # Bouton pour appliquer les filtres
    bouton_apply = widgets.Button(description="Appliquer les filtres", button_style='success')

    # Fonction pour récupérer les filtres et exécuter l'analyse
    def appliquer_filtres(_):
        selected_analyses = list(analyse_select.value)
        
        # Exécuter chaque analyse sélectionnée
        for analyse in selected_analyses:
            func = analyses_disponibles[analyse]
            func(df, departement_select, type_bien_select,tranche_prix_select,tranche_surface_select)
    
    # Lier le bouton à la fonction
    bouton_apply.on_click(appliquer_filtres)
    
    # Afficher les widgets
    display(departement_select, type_bien_select,tranche_prix_select,tranche_surface_select, analyse_select, bouton_apply)

In [133]:
def afficher_prix_moyen_par_m2(df,departement_select,type_bien_select,tranche_prix_select,tranche_surface_select):

    # Récupérer les départements et types de bien sélectionnés
    selected_departements = list(departement_select.value)
    selected_types_bien = list(type_bien_select.value)
    selected_tranches_prix = list(tranche_prix_select.value)
    selected_tranche_surface = list(tranche_surface_select.value)

    # Si rien n'est sélectionné, on prend tous les départements et types de biens
    if not selected_departements:
        selected_departements = list(df['code_departement'].unique())
    if not selected_types_bien:
        selected_types_bien = list(df['type_local'].unique())
    if not selected_tranches_prix:
        selected_tranches_prix = list(df['tranche_valeur'].unique())
    if not selected_tranche_surface:
        selected_tranche_surface = list(df['plage_surface'].unique())
    
    # Filtrer les données en fonction des départements et types de bien sélectionnés
    df_filtered = df[(df['code_departement'].isin(selected_departements)) & 
                        (df['type_local'].isin(selected_types_bien))& 
                        (df['tranche_valeur'].isin(selected_tranches_prix))& 
                        (df['plage_surface'].isin(selected_tranche_surface))].copy()

    # Calculer le prix moyen par m² par département
    prix_moyen_par_dept = df_filtered.groupby('code_departement')['prix_m2'].mean()

    # Calculer la moyenne des prix des départements sélectionnés
    prix_moyen_selectionne = prix_moyen_par_dept.mean()

    # Calculer la moyenne nationale des prix par m²
    prix_moyen_national = df[df['type_local'].isin(selected_types_bien)].groupby('code_departement')['prix_m2'].mean().mean()

    # Création du graphique
    fig, ax = plt.subplots(figsize=(20, 6))

    # Tracer le prix moyen pour chaque département sélectionné
    ax.bar(prix_moyen_par_dept.index, prix_moyen_par_dept)

    # Ajouter une barre pour la moyenne des départements sélectionnés
    ax.bar('Moyenne Sélectionnée', prix_moyen_selectionne, alpha=0.7)

    # Ajouter une barre pour la moyenne nationale
    ax.bar('Moyenne Nationale', prix_moyen_national, alpha=0.7)

    # Paramètres du graphique
    ax.set_ylabel('Prix moyen par m² (€)')
    ax.set_xlabel('Départements')
    ax.set_title("Prix moyen par m² par département, sélection et moyenne globale")
    ax.grid(True, linestyle='--', alpha=0.6)
    plt.xticks(rotation=90)
    plt.tight_layout()  # Pour éviter que les labels se chevauchent
    plt.show()


In [134]:
def afficher_surface_moyenne_par_dept(df,departement_select,type_bien_select,tranche_prix_select,tranche_surface_select):
    
    # Récupérer les départements et types de bien sélectionnés
    selected_departements = list(departement_select.value)
    selected_types_bien = list(type_bien_select.value)
    selected_tranches_prix = list(tranche_prix_select.value)
    selected_tranche_surface = list(tranche_surface_select.value)

    # Si rien n'est sélectionné, on prend tous les départements et types de biens
    if not selected_departements:
        selected_departements = list(df['code_departement'].unique())
    if not selected_types_bien:
        selected_types_bien = list(df['type_local'].unique())
    if not selected_tranches_prix:
        selected_tranches_prix = list(df['tranche_valeur'].unique())
    if not selected_tranche_surface:
        selected_tranche_surface = list(df['plage_surface'].unique())
    
    # Filtrer les données en fonction des départements et types de bien sélectionnés
    df_filtered = df[(df['code_departement'].isin(selected_departements)) & 
                        (df['type_local'].isin(selected_types_bien))& 
                        (df['tranche_valeur'].isin(selected_tranches_prix))& 
                        (df['plage_surface'].isin(selected_tranche_surface))].copy()

    
    # Calculer la surface moyenne pour chaque département sélectionné
    surface_moyenne_par_dept = df_filtered.groupby('code_departement')['surface_terrain'].mean()

    # Calculer la moyenne des surfaces des départements sélectionnés
    surface_moyenne_selectionnee = surface_moyenne_par_dept.mean()

    # Calculer la surface moyenne nationale (moyenne de tous les départements et types de bien sélectionnés)
    surface_moyenne_nationale = df[df['type_local'].isin(selected_types_bien)].groupby('code_departement')['surface_terrain'].mean().mean()

    # Création du graphique
    fig, ax = plt.subplots(figsize=(20, 6))

    # Tracer la surface moyenne pour chaque département sélectionné
    ax.bar(surface_moyenne_par_dept.index, surface_moyenne_par_dept, label='Surface Moyenne par Département')

    # Ajouter une barre pour la moyenne des départements sélectionnés
    ax.bar('Moyenne Sélectionnée', surface_moyenne_selectionnee, label='Moyenne Sélectionnée', alpha=0.7)

    # Ajouter une barre pour la moyenne nationale
    ax.bar('Moyenne Nationale', surface_moyenne_nationale, label='Moyenne Nationale', alpha=0.7)

    # Paramètres du graphique
    ax.set_ylabel('Surface moyenne en m²')
    ax.set_xlabel('Départements')
    ax.set_title("Surface moyenne en m² par département, sélection et moyenne globale")
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.legend(title="Légende", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=90)
    plt.tight_layout()  # Pour éviter que les labels se chevauchent
    plt.show()

In [135]:
def evolution_prix_moyen_par_m2_par_departement_mensuel(df,departement_select,type_bien_select,tranche_prix_select,tranche_surface_select):
    # Récupérer les départements et types de bien sélectionnés
    selected_departements = list(departement_select.value)
    selected_types_bien = list(type_bien_select.value)
    selected_tranches_prix = list(tranche_prix_select.value)
    selected_tranche_surface = list(tranche_surface_select.value)

    # Si rien n'est sélectionné, on prend tous les départements et types de biens
    if not selected_departements:
        selected_departements = list(df['code_departement'].unique())
    if not selected_types_bien:
        selected_types_bien = list(df['type_local'].unique())
    if not selected_tranches_prix:
        selected_tranches_prix = list(df['tranche_valeur'].unique())
    if not selected_tranche_surface:
        selected_tranche_surface = list(df['plage_surface'].unique())
    
    # Filtrer les données en fonction des départements et types de bien sélectionnés
    df_filtered = df[(df['code_departement'].isin(selected_departements)) & 
                        (df['type_local'].isin(selected_types_bien))& 
                        (df['tranche_valeur'].isin(selected_tranches_prix))& 
                        (df['plage_surface'].isin(selected_tranche_surface))].copy()

    # S'assurer que la colonne de date est au bon format (si ce n'est pas déjà le cas)
    df_filtered['date_mutation'] = pd.to_datetime(df_filtered['date_mutation'])

    # Groupement par département et par mois
    prix_moyen_par_dept_mois = df_filtered.groupby(['code_departement', 'mois_annee'])['prix_m2'].mean().reset_index()

    # Créer le graphique
    plt.figure(figsize=(15, 8))

    # Ajouter la courbe pour la moyenne générale
    df_dept_global = df_filtered.groupby('mois_annee')['prix_m2'].mean().reset_index()
    plt.plot(df_dept_global['mois_annee'].astype(str), df_dept_global['prix_m2'], marker='o', label='Moyenne générale', linewidth=2)

    # Ajouter la courbe pour la moyenne nationale (basée sur l'intégralité des données)
    df_national_global = df_filtered.groupby('mois_annee')['prix_m2'].mean().reset_index()
    plt.plot(df_national_global['mois_annee'].astype(str), df_national_global['prix_m2'], marker='o', label='Moyenne nationale', linewidth=2)

    # Ajouter les courbes pour chaque département sélectionné
    for departement in selected_departements:
        df_dept = prix_moyen_par_dept_mois[prix_moyen_par_dept_mois['code_departement'] == departement]
        plt.plot(df_dept['mois_annee'].astype(str), df_dept['prix_m2'], marker='o', label=f'Département {departement}')

    plt.title("Évolution du prix moyen par m² par département (mensuel)")
    plt.xlabel("Mois et Année")
    plt.ylabel("Prix moyen par m² (€)")
    plt.legend(title="Types de Bien", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xticks(rotation=45)
    plt.tight_layout()  # Pour éviter que les labels se chevauchent
    plt.show()

In [136]:
def afficher_nombre_piece_departement(df,departement_select,type_bien_select,tranche_prix_select,tranche_surface_select):
    # Récupérer les départements et types de bien sélectionnés
    selected_departements = list(departement_select.value)
    selected_types_bien = list(type_bien_select.value)
    selected_tranches_prix = list(tranche_prix_select.value)
    selected_tranche_surface = list(tranche_surface_select.value)

    # Si rien n'est sélectionné, on prend tous les départements et types de biens
    if not selected_departements:
        selected_departements = list(df['code_departement'].unique())
    if not selected_types_bien:
        selected_types_bien = list(df['type_local'].unique())
    if not selected_tranches_prix:
        selected_tranches_prix = list(df['tranche_valeur'].unique())
    if not selected_tranche_surface:
        selected_tranche_surface = list(df['plage_surface'].unique())
    
    # Filtrer les données en fonction des départements et types de bien sélectionnés
    df_filtered = df[(df['code_departement'].isin(selected_departements)) & 
                        (df['type_local'].isin(selected_types_bien))& 
                        (df['tranche_valeur'].isin(selected_tranches_prix))& 
                        (df['plage_surface'].isin(selected_tranche_surface))].copy()

    # Groupement par département pour calculer la moyenne des pièces
    nombre_piece_par_dept = df_filtered.groupby('code_departement')['nombre_pieces_principales'].mean().reset_index()

    # Ajouter la moyenne générale des départements sélectionnés
    moyenne_selection = df_filtered['nombre_pieces_principales'].mean()
    moyenne_nationale = df['nombre_pieces_principales'].mean()

    # Ajouter une barre pour la moyenne générale
    departements = list(nombre_piece_par_dept['code_departement'])
    nombre_piece = list(nombre_piece_par_dept['nombre_pieces_principales'])

    # Créer le graphique
    plt.figure(figsize=(15, 8))
    plt.bar(departements, nombre_piece, color='skyblue')
    plt.axhline(moyenne_selection, color='red', linestyle='dashed', linewidth=2, label='Moyenne Sélectionnée')
    plt.axhline(moyenne_nationale, color='green', linestyle='dashed', linewidth=2, label='Moyenne nationale')

    plt.title("Nombre moyen de pièces par département et par type de bien sélectionné")
    plt.xlabel("Département")
    plt.ylabel("Nombre moyen de pièces")
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()  # Pour éviter que les labels se chevauchent
    plt.show()

In [137]:
def repartition_type_bien_par_departement(df,departement_select,type_bien_select,tranche_prix_select,tranche_surface_select):
    # Récupérer les départements et types de bien sélectionnés
    selected_departements = list(departement_select.value)
    selected_types_bien = list(type_bien_select.value)
    selected_tranches_prix = list(tranche_prix_select.value)
    selected_tranche_surface = list(tranche_surface_select.value)

    # Si rien n'est sélectionné, on prend tous les départements et types de biens
    if not selected_departements:
        selected_departements = list(df['code_departement'].unique())
    if not selected_types_bien:
        selected_types_bien = list(df['type_local'].unique())
    if not selected_tranches_prix:
        selected_tranches_prix = list(df['tranche_valeur'].unique())
    if not selected_tranche_surface:
        selected_tranche_surface = list(df['plage_surface'].unique())
    
    # Filtrer les données en fonction des départements et types de bien sélectionnés
    df_filtered = df[(df['code_departement'].isin(selected_departements)) & 
                        (df['type_local'].isin(selected_types_bien))& 
                        (df['tranche_valeur'].isin(selected_tranches_prix))& 
                        (df['plage_surface'].isin(selected_tranche_surface))].copy()

    # Calculer la répartition des types de biens par département
    repartition = df_filtered.groupby(['code_departement', 'type_local']).size().unstack(fill_value=0)

    # Normaliser pour obtenir des proportions (en %)
    repartition_norm = repartition.div(repartition.sum(axis=1), axis=0) * 100

    # Création d'un graphique en barres empilées
    repartition_norm.plot(kind='bar', figsize=(15, 8), stacked=True, colormap='tab20')

    #  option d'affichage du graphique
    plt.title("Répartition des transactions par type de bien et par département")
    plt.xlabel("Département")
    plt.ylabel("Proportion des transactions (%)")
    plt.legend(title="Type de bien", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.show()

In [138]:
def repartition_Valeur_fonciere_par_tranche(df,departement_select,type_bien_select,tranche_prix_select,tranche_surface_select):
    # Récupérer les départements et types de bien sélectionnés
    selected_departements = list(departement_select.value)
    selected_types_bien = list(type_bien_select.value)
    selected_tranches_prix = list(tranche_prix_select.value)
    selected_tranche_surface = list(tranche_surface_select.value)

    # Si rien n'est sélectionné, on prend tous les départements et types de biens
    if not selected_departements:
        selected_departements = list(df['code_departement'].unique())
    if not selected_types_bien:
        selected_types_bien = list(df['type_local'].unique())
    if not selected_tranches_prix:
        selected_tranches_prix = list(df['tranche_valeur'].unique())
    if not selected_tranche_surface:
        selected_tranche_surface = list(df['plage_surface'].unique())
    
    # Filtrer les données en fonction des départements et types de bien sélectionnés
    df_filtered = df[(df['code_departement'].isin(selected_departements)) & 
                        (df['type_local'].isin(selected_types_bien))& 
                        (df['tranche_valeur'].isin(selected_tranches_prix))& 
                        (df['plage_surface'].isin(selected_tranche_surface))].copy()
    # Compter le nombre de transactions par tranche et par département
    repartition = df_filtered.groupby(['code_departement', 'tranche_valeur']).size().unstack(fill_value=0)

    # Normaliser pour obtenir des proportions (en %)
    repartition_norm = repartition.div(repartition.sum(axis=1), axis=0) * 100

    # Création du graphique
    repartition_norm.T.plot(kind='bar', figsize=(15, 8), stacked=True, colormap='tab10')

    # option d'affichage du graphique
    plt.title("Répartition des transactions par tranche de valeur foncière et par département")
    plt.xlabel("Tranche de valeur foncière (€)")
    plt.ylabel("Proportion des transactions (%)")
    plt.legend(title="Département", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.show()

In [139]:
def prix_m2_par_surface(df, departement_select, type_bien_select,tranche_prix_select,tranche_surface_select):
    # Récupérer les départements et types de bien sélectionnés
    selected_departements = list(departement_select.value)
    selected_types_bien = list(type_bien_select.value)
    selected_tranches_prix = list(tranche_prix_select.value)
    selected_tranche_surface = list(tranche_surface_select.value)

    # Si rien n'est sélectionné, on prend tous les départements et types de biens
    if not selected_departements:
        selected_departements = list(df['code_departement'].unique())
    if not selected_types_bien:
        selected_types_bien = list(df['type_local'].unique())
    if not selected_tranches_prix:
        selected_tranches_prix = list(df['tranche_valeur'].unique())
    if not selected_tranche_surface:
        selected_tranche_surface = list(df['plage_surface'].unique())
    
    # Filtrer les données en fonction des départements et types de bien sélectionnés
    df_filtered = df[(df['code_departement'].isin(selected_departements)) & 
                        (df['type_local'].isin(selected_types_bien))& 
                        (df['tranche_valeur'].isin(selected_tranches_prix))& 
                        (df['plage_surface'].isin(selected_tranche_surface))].copy()

    # Calculer le prix moyen par plage de surface et département
    prix_moyen_par_surface = df_filtered.groupby(['plage_surface', 'code_departement'])['prix_m2'].mean().reset_index()
    
     # Calculer la moyenne par plage de surface pour la sélection
    moyenne_selection = df_filtered.groupby('plage_surface')['prix_m2'].mean().reset_index()

    # Calculer la moyenne nationale par plage de surface
    moyenne_nationale = df.groupby('plage_surface')['prix_m2'].mean().reset_index()

    # Création du graphique
    plt.figure(figsize=(12, 8))

    for departement in selected_departements:
        df_departement = prix_moyen_par_surface[prix_moyen_par_surface['code_departement'] == departement]
        if not df_departement.empty:
            plt.plot(
                df_departement['plage_surface'], 
                df_departement['prix_m2'], 
                marker='o', 
                label=f'Département {departement}'
            )
    plt.plot(
        moyenne_selection['plage_surface'], 
        moyenne_selection['prix_m2'], 
        marker='o', 
        linestyle='--', 
        label='Moyenne sélectionnée', 
        color='black'
    )

    # Ajouter la courbe de la moyenne nationale
    plt.plot(
        moyenne_nationale['plage_surface'], 
        moyenne_nationale['prix_m2'], 
        marker='o', 
        linestyle=':', 
        label='Moyenne nationale', 
        color='red'
    )

    # Configurations du graphique
    plt.title("Prix moyen par m² en fonction des plages de surface")
    plt.xlabel("Plage de surface (m²)")
    plt.ylabel("Prix moyen par m² (€)")
    plt.legend(title="Départements", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [140]:
def afficher_carte_france(data, departement_select, type_bien_select, tranche_prix_select, tranche_surface_select):
    chemin_fichier_shp = "regions.shx"
    metropolitan_departments = [f"{i:02}" for i in range(1, 96)]+["2A","2B"]

    selected_departements = list(departement_select.value)
    selected_types_bien = list(type_bien_select.value)
    selected_tranches_prix = list(tranche_prix_select.value)
    selected_tranche_surface = list(tranche_surface_select.value)

    # Si rien n'est sélectionné, on prend tous les départements et types de biens
    if not selected_departements:
        selected_departements = list(data['code_departement'].unique())
    if not selected_types_bien:
        selected_types_bien = list(data['type_local'].unique())
    if not selected_tranches_prix:
        selected_tranches_prix = list(data['tranche_valeur'].unique())
    if not selected_tranche_surface:
        selected_tranche_surface = list(data['plage_surface'].unique())

    # Filtrer les données en fonction des départements et types de bien sélectionnés
    data_filtered = data[
        (data['code_departement'].isin(metropolitan_departments)) &
        (data['code_departement'].isin(selected_departements)) &
        (data['type_local'].isin(selected_types_bien)) &
        (data['tranche_valeur'].isin(selected_tranches_prix)) &
        (data['plage_surface'].isin(selected_tranche_surface))
    ].copy()

    # Charger le fichier shapefile
    carte_france = gpd.read_file(chemin_fichier_shp)

    # Normaliser les prix pour les échelles de couleurs
    data_filtered['prix_normalise'] = (data_filtered['valeur_fonciere'] - data_filtered['valeur_fonciere'].min()) / (
        data_filtered['valeur_fonciere'].max() - data_filtered['valeur_fonciere'].min()
    )

    # Tracer la carte
    plt.figure(figsize=(100, 100))
    carte_france.plot(edgecolor='black', facecolor='lightgray', alpha=0.7)

    # Ajouter les points correspondants aux ventes filtrées avec les couleurs
    scatter = plt.scatter(
        data_filtered['longitude'],
        data_filtered['latitude'],
        c=data_filtered['prix_normalise'],  # Utiliser directement les valeurs normalisées
        cmap='RdYlGn',  # Couleur de vert à rouge
        s=1,  # Taille des points
        label="Ventes filtrées"
    )

    # Configurations du graphique
    plt.title("Carte de France")
    plt.xlim(-5, 10)
    plt.ylim(41, 52)

    # Ajouter une barre de couleur liée au scatter plot
    cbar = plt.colorbar(scatter, label="Prix relatif (bas à haut)")

    plt.legend(loc="lower left")
    plt.show()


### III. Exécution des fonctions

In [141]:
selection_filtres_et_analyses(data)

SelectMultiple(description='Départements', layout=Layout(width='60%'), options=('01', '02', '03', '04', '05', …

SelectMultiple(description='Types de biens', layout=Layout(width='60%'), options=('Maison', 'Dépendance (isolé…

SelectMultiple(description='Tranche de prix', layout=Layout(width='60%'), options=('100k - 200k €', '200k - 30…

SelectMultiple(description='Tranche surface', layout=Layout(width='60%'), options=('>500 m²', '200-500 m²', '5…

SelectMultiple(description='Analyse', layout=Layout(width='60%'), options=('Prix moyen par m²', 'Surface moyen…

Button(button_style='success', description='Appliquer les filtres', style=ButtonStyle())